In [ ]:
!pip install flask-ngrok

# Train MLP

In [ ]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from sklearn.preprocessing import MinMaxScaler

In [ ]:
!pip list | grep tensorflow

tensorflow                    2.5.0              
tensorflow-datasets           4.0.1              
tensorflow-estimator          2.5.0              
tensorflow-gcs-config         2.5.0              
tensorflow-hub                0.12.0             
tensorflow-metadata           1.1.0              
tensorflow-probability        0.13.0             


## preprocess data

In [ ]:
# load data
X = pd.read_csv('X.csv')

with open('y.npy', 'rb') as f:
    y = np.load(f)

# OverallQual 10
# GrLivArea 10
# GarageCars 9
# GarageArea 8
# TotalBsmtSF 7
# 1stFlrSF 6
# FullBath 5
# LotShape Reg
X = X[['OverallQual', 'GrLivArea', 'GarageCars', 'GarageArea', 'TotalBsmtSF', '1stFlrSF', 'FullBath', 'LotShape_rank']]

x_min_max_scaler = MinMaxScaler()
x_min_max_scaler.fit(X)
scaled_X = x_min_max_scaler.transform(X)

y_min_max_scaler = MinMaxScaler()
y_min_max_scaler.fit(y)
scaled_y = y_min_max_scaler.transform(y)

## training

In [ ]:
model = keras.Sequential(
      [
          keras.Input(shape=scaled_X.shape[-1]),
          layers.Dense(96, activation='relu'),
          layers.Dense(48, activation='relu'),
          layers.Dense(1)
      ]
  )

model.compile(loss="mse", optimizer="adam")

early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15)
model.fit(scaled_X, scaled_y, 
          batch_size=2, epochs=150, 
          callbacks=[early_stopping_callback], validation_split=0.05)

Epoch 1/150
694/694 [==============================] - 2s 2ms/step - loss: 0.0044 - val_loss: 0.0015
Epoch 2/150
694/694 [==============================] - 1s 1ms/step - loss: 0.0034 - val_loss: 0.0023
Epoch 3/150
694/694 [==============================] - 1s 1ms/step - loss: 0.0033 - val_loss: 0.0015
Epoch 4/150
694/694 [==============================] - 1s 1ms/step - loss: 0.0030 - val_loss: 0.0016
Epoch 5/150
694/694 [==============================] - 1s 1ms/step - loss: 0.0027 - val_loss: 0.0066
Epoch 6/150
694/694 [==============================] - 1s 1ms/step - loss: 0.0029 - val_loss: 0.0016
Epoch 7/150
694/694 [==============================] - 1s 1ms/step - loss: 0.0027 - val_loss: 0.0020
Epoch 8/150
694/694 [==============================] - 1s 1ms/step - loss: 0.0027 - val_loss: 0.0015
Epoch 9/150
694/694 [==============================] - 1s 1ms/step - loss: 0.0026 - val_loss: 0.0025
Epoch 10/150
694/694 [==============================] - 1s 1ms/step - loss: 0.0025 - val_lo

In [ ]:
pred = model.predict(scaled_X[:1]) # 0 ~ 1
pred = y_min_max_scaler.inverse_transform(pred)

## Save MLP model

In [ ]:
#앞서 학습시킨 모델(adam형 compile)을, 학습시키지 않고, 바로 모델을 저장하고 load해서 쓰고자할때
#학습된 모델을, h5라는 형태로 저장하고, load_model 통해 그대로 쓸 수 있음
model.save("mlp_v0.1.h5")

In [ ]:
reconstructed_model = keras.models.load_model("mlp_v0.1.h5")

In [ ]:
pred = reconstructed_model.predict(scaled_X[:1]) # 0 ~ 1
pred = y_min_max_scaler.inverse_transform(pred)

In [ ]:
pred

array([[201507.44]], dtype=float32)

# Flask server

In [ ]:
from flask import Flask, render_template, request
from flask_ngrok import run_with_ngrok

# load data
X = pd.read_csv('X.csv')

with open('y.npy', 'rb') as f:
    y = np.load(f)

# OverallQual 10
# GrLivArea 10
# GarageCars 9
# GarageArea 8
# TotalBsmtSF 7
# 1stFlrSF 6
# FullBath 5
# LotShape Reg
X = X[['OverallQual', 'GrLivArea', 'GarageCars', 'GarageArea', 'TotalBsmtSF', '1stFlrSF', 'FullBath', 'LotShape_rank']]

x_min_max_scaler = MinMaxScaler()
x_min_max_scaler.fit(X)

y_min_max_scaler = MinMaxScaler()
y_min_max_scaler.fit(y)

# load model
reconstructed_model = keras.models.load_model("mlp_v0.1.h5")

# run server
app = Flask(__name__, template_folder='/content')
run_with_ngrok(app)

def preprocess_data(data):
  # return np.zeros((1, 8)) # dummy data

  """
  Dictionary --> np array (1, 8)

  OverallQual 2
  GrLivArea 5000
  GarageCars 2
  GarageArea 480
  TotalBsmtSF 991
  1stFlrSF 1087
  FullBath 2
  LotShape IR3 --> 1, 2, 3, 4
  """
  X = [] # <-- OverallQual, GrLivArea, ... LotShape
  for k, v in data.items():
    if k == 'LotShape':
      if v == 'Reg':
        X.append(4)
      elif v == 'IR1':
        X.append(3)
      elif v == 'IR2':
        X.append(2)
      elif v == 'IR1':
        X.append(1)
    else:
      X.append(float(v))

  # X = [2, 5000, 2, ... , 3]
  X = np.array(X) # (8,)
  X = X.reshape((1, -1)) # (1, 8)

  # min max scaling
  scaled_X = x_min_max_scaler.transform(X)

  return scaled_X
  
@app.route("/")
def predict():
  #return "<h1>This is your Flask server.</h1>"
  return render_template("submit_form.html")

@app.route("/result", methods=['POST'])
def result():
  # Read data [v]
  # Proprocess data 
  # Model prediction
  # Return prediction

  data = request.form # User data

  message = ""
  message += "<h1>House Price</h1>"

  for k, v in data.items():
    print(k, v)
    message += k + ": " + v + "</br>"

  # 데이터 전처리
  X = preprocess_data(data) # User data --> (1, 8) array

  pred = reconstructed_model.predict(X)
  pred = y_min_max_scaler.inverse_transform(pred)
  # array (1, 1) --> string

  message += "</br>"
  message += "Predicted price: " + str(pred[0][0])

  return message
    
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://4b61824c02ca.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [18/Jul/2021 07:42:41] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Jul/2021 07:42:41] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [18/Jul/2021 07:42:43] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Jul/2021 07:42:45] "POST /result HTTP/1.1" 200 -


OverallQual 6
GrLivArea 1464
GarageCars 2
GarageArea 480
TotalBsmtSF 991
1stFlrSF 1087
FullBath 2
LotShape IR1


127.0.0.1 - - [18/Jul/2021 07:42:49] "POST /result HTTP/1.1" 200 -


OverallQual 10
GrLivArea 1464
GarageCars 2
GarageArea 480
TotalBsmtSF 991
1stFlrSF 1087
FullBath 2
LotShape IR1


127.0.0.1 - - [18/Jul/2021 07:42:56] "POST /result HTTP/1.1" 200 -


OverallQual 10
GrLivArea 5000
GarageCars 2
GarageArea 480
TotalBsmtSF 991
1stFlrSF 1087
FullBath 2
LotShape IR1


127.0.0.1 - - [18/Jul/2021 07:43:07] "GET /result HTTP/1.1" 405 -
